Adapted from the tutorial on UN Spider site  
The original UN tutorial contains more detail and analysis  
https://un-spider.org/advisory-support/recommended-practices/recommended-practice-google-earth-engine-flood-mapping/step-by-step  
Converted to Python using geemap  
Author of this notebook: oliverburdekin@burdgis.com

In [72]:
import os
import ee
import geemap

Map = geemap.Map()
ee.Initialize()

In [97]:
# Define AOI geom

# # Beira, Mozambique
# geometry = ee.Geometry.Polygon([
#     [
#         [35.53377589953368, -19.6674648789114],
#         [34.50106105578368, -18.952058786515526],
#         [33.63314113390868, -19.87423907259203],
#         [34.74825343859618, -20.61123742951084]
#     ]
# ])

# Fish Lake, Doncaster
geometry = ee.Geometry.Polygon([
    
    [
        [-1.317268, 53.790878],
        [-0.906274, 53.790878],
        [-0.906274, 53.452582],
        [-1.317268, 53.452582]
    ]
])


In [106]:
# Define dates (before and after flooding)
before_start= '2019-10-30'
before_end='2019-11-09'

after_start='2019-11-13'
after_end='2019-11-15'

# # Beira dates
# before_start = '2019-03-01'
# before_end='2019-03-10'

# # Now set the same parameters for AFTER the flood.
# after_start='2019-03-10'
# after_end='2019-03-23'

In [99]:
#*******************************************************************************************
# SET SAR PARAMETERS (can be left default)#

polarization = "VH" #or 'VV' --> VH mostly is the prefered polarization for flood mapping.
                           # However, it always depends on your study area, you can select 'VV'
                           # as well.#
pass_direction = "DESCENDING" # or 'ASCENDING'when images are being compared use only one
                           # pass direction. Consider changing this parameter, if your image
                           # collection is empty. In some areas more Ascending images exist than
                           # than descending or the other way around.#
difference_threshold = 1.25 #threshodl to be applied on the difference image (after flood
#                            - before flood). It has been chosen by trial and error. In case your
#                            flood extent result shows many False-positive or negative signals,
#                            consider changing it! #
#relative_orbit = 79
                          # #if you know the relative orbit for your study area, you can filter
                          #  you image collection by it here, to avoid errors caused by comparing
                          #  different relative orbits.#

In [100]:
# rename selected geometry feature
aoi = ee.FeatureCollection(geometry)

# Load and filter Sentinel-1 GRD data by predefined parameters
collection= ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.eq('instrumentMode','IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
  .filter(ee.Filter.eq('orbitProperties_pass',pass_direction)) \
  .filter(ee.Filter.eq('resolution_meters',10)) \
  .filterBounds(aoi) \
  .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start,after_end)

In [101]:
def dates(imgcol):
    range = imgcol.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
    printed = ee.String('from ') \
          .cat(ee.Date(range.get('min')).format('YYYY-MM-dd')) \
          .cat(' to ') \
          .cat(ee.Date(range.get('max')).format('YYYY-MM-dd'))
    return printed

In [102]:
# Create a mosaic of selected tiles and clip to study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply reduce the radar speckle by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')


In [103]:
# Calculate the difference between before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

In [104]:
# Refine flood result with additional datasets

# JRC layer on surface water seasonality
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))

#Flooded layer where perennial water bodies (water > 10 mo/yr) is assigned a 0 value
flooded_mask = difference_binary.where(swater_mask,0)
# final flooded area without pixels in perennial waterbodies
flooded = flooded_mask.updateMask(flooded_mask)

# Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
# This operation reduces noise of the flood extent product
connections = flooded.connectedPixelCount()
flooded2 = flooded.updateMask(connections.gte(8))

In [107]:
Map.setCenter(-1.012291,53.611961,12)

# Before and after layers
Map.addLayer(before_filtered, {'min':-25, 'max':0}, 'Before Flood',0)
Map.addLayer(after_filtered, {'min':-25, 'max':0}, 'After Flood',1)

# Difference layer
Map.addLayer(difference,{'min':0, 'max':2},"Difference Layer",0)

# Flooded areas
Map.addLayer(flooded,{'palette':"0000FF"},'Flooded areas')
Map.addLayer(flooded2,{'palette':"0000FF"},'Flooded areas 2')

Map

Map(bottom=2520.0, center=[53.611961, -1.012291], controls=(WidgetControl(options=['position'], widget=HBox(ch…